# lab-10_2_mnist_deep_cnn
##### JaehyunJeong

In [24]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

from torchsummary import summary

In [25]:
# conv 연산 연습
conv = nn.Conv2d(1,1,11,stride=4, padding=0) # 입력채널, 출력채널(=필터갯수), 필터사이즈
print("conv 연산 : ",conv)

inputs = torch.Tensor(1,1,227,227) # 배치사이즈, 채널, 높이, 너비
print("입력 이미지 크기 : ",inputs.shape)

out = conv(inputs)
print("출력 이미지 크기 : ",out.shape)

conv 연산 :  Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))
입력 이미지 크기 :  torch.Size([1, 1, 227, 227])
출력 이미지 크기 :  torch.Size([1, 1, 55, 55])


In [26]:
# conv + pooling
inputs = torch.Tensor(1,1,28,28)
conv = nn.Conv2d(1,5,5)
pool = nn.MaxPool2d(2) # 풀링 계층의 필터 사이즈는 2x2

out = conv(inputs)
out2 = pool(out)

out.size()

torch.Size([1, 5, 24, 24])

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)

In [28]:
device

'cpu'

In [29]:
# parameter
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [30]:
#mnist dataset

mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                          train = False,
                          transform = transforms.ToTensor(),
                          download=True)

In [31]:
# loader 생성
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [32]:
# 모델 생성
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        # conv layer
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1), # 입력 채널 1, 출력 채널(필터 갯수) 32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # conv layer
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # conv layer
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )

        self.fc1 = nn.Linear(4*4*128, 625, bias=True) # 4*4 사이즈의 맵이 64개(입력), 625(출력)
        torch.nn.init.xavier_uniform_(self.fc1.weight) # fc층은 가중치 초기화 적용

        # fc layer
        self.layer4 = nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = self.keep_prob)
        )

        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x) # conv
        out = self.layer2(out) # conv
        out = self.layer3(out) # conv
        out = out.view(out.size(0), -1) # flatten
        out = self.layer4(out) # fc
        out = self.fc2(out) # fc
        return out

In [33]:
model = CNN().to(device)
summary(model, (1,28,28), batch_size=batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [100, 32, 28, 28]             320
              ReLU-2          [100, 32, 28, 28]               0
         MaxPool2d-3          [100, 32, 14, 14]               0
            Conv2d-4          [100, 64, 14, 14]          18,496
              ReLU-5          [100, 64, 14, 14]               0
         MaxPool2d-6            [100, 64, 7, 7]               0
            Conv2d-7           [100, 128, 7, 7]          73,856
              ReLU-8           [100, 128, 7, 7]               0
         MaxPool2d-9           [100, 128, 4, 4]               0
           Linear-10                 [100, 625]       1,280,625
           Linear-11                 [100, 625]       1,280,625
             ReLU-12                 [100, 625]               0
          Dropout-13                 [100, 625]               0
           Linear-14                  [

In [34]:
# 파라미터 정의
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [35]:
# training
total_batch = len(data_loader) # 600개의 배치 묶음
model.train() # dropout = True

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader : # (image, label)
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y) # 한 배치묶음(100개의 이미지)의 평균 loss(cost)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch : {:>2}] cost = {:.9}'.format(epoch+1, avg_cost))

print('Learning Finished!')

[Epoch :  1] cost = 0.196370274
[Epoch :  2] cost = 0.0524963289
[Epoch :  3] cost = 0.0392724425
[Epoch :  4] cost = 0.0289225206
[Epoch :  5] cost = 0.022951914
[Epoch :  6] cost = 0.0187023841
[Epoch :  7] cost = 0.0173200276
[Epoch :  8] cost = 0.0152993798
[Epoch :  9] cost = 0.0118491845
[Epoch : 10] cost = 0.0129938331
[Epoch : 11] cost = 0.010562093
[Epoch : 12] cost = 0.00931800716
[Epoch : 13] cost = 0.00956155546
[Epoch : 14] cost = 0.00598781416
[Epoch : 15] cost = 0.00741908466
Learning Finished!


In [52]:
# test model and check accuracy
with torch.no_grad():
  model.eval() # dropout = False

  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28,28).float().to(device) # 배치사이즈*
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('accuracy : {}'.format(accuracy.item()))

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


accuracy : 0.9887999892234802
